# Type fields (refs) parsing

Attempt to produce the info of `data.py` by parsing the C++ in `raw_header.h`.  
This is very quick and dirty code, you have been warned.

In [273]:
import re
f = open('art-sdk/sdk/runtime/vm/raw_object.h')

entries = []

def try_to_parse_snapshot(f, l):
    pref = '  RawObject** to_snapshot(Snapshot::Kind kind) {'
    if not l.startswith(pref): return False
    print('  parsing snapshot')
    code = l[len(pref):]
    outcode = ''
    lev = 0
    while True:
        if len(code):
            c = code[0]
            code = code[1:]
        else:
            c = f.read(1)
        if c == '{': lev += 1
        elif c == '}':
            if lev == 0: break
            lev -= 1
        outcode += c
    entry['snapshot'] = outcode
    return True

last_class = None
while True:
    l = f.readline()
    if not l: break

    m = re.match('class (.+) :', l)
    if m:
        print('class', m.group(1))
        last_class = m.group(1)
        entry = None
        continue

    m = re.match(r'  VISIT_FROM\((.+), (.+)\)', l)
    if m:
        fi = m.group(2)
        print('  visit from', fi)
        if fi == 'names()[0]': fi = 'names_'
        assert last_class.startswith('Raw') and fi.endswith('_')
        entry = { 'kind': last_class[3:], 'from': fi[:-1] }
        
        fields = []
        while True:
            l = f.readline()
            m = re.match(r'  VISIT_TO(_LENGTH)?\((.+), (.+)\)', l)
            if m:
                #fi = m.group(2)
                print('  visit to')
                #assert fi.endswith('_')
                #entry['to'] = fi[:-1]
                break
            if try_to_parse_snapshot(f, l): continue
            if l.startswith('#'): entry['has_preproc'] = True
            fields.append(l)
        entry['fields'] = fields
        entries.append(entry)
    
    if try_to_parse_snapshot(f, l): continue

class RawClass
  visit from name_
  visit to
  parsing snapshot
class RawPatchClass
  visit from patched_class_
  visit to
  parsing snapshot
class RawFunction
  visit from name_
  parsing snapshot
  visit to
class RawClosureData
  visit from context_scope_
  visit to
class RawSignatureData
  visit from parent_function_
  visit to
  parsing snapshot
class RawRedirectionData
  visit from type_
  visit to
  parsing snapshot
class RawFfiTrampolineData
  visit from signature_type_
  visit to
  parsing snapshot
class RawField
  visit from name_
  parsing snapshot
  visit to
class RawScript
  visit from url_
  visit to
  parsing snapshot
class RawLibrary
  visit from name_
  parsing snapshot
  visit to
class RawNamespace
  visit from library_
  visit to
  parsing snapshot
class RawKernelProgramInfo
  visit from string_offsets_
  visit to
  parsing snapshot
class RawCode
  visit from object_pool_
  visit to
class RawBytecode
  visit from object_pool_
  visit to
  parsing snapshot
class RawObj

In [274]:
snapshot_matcher = \
    re.escape('switch (kind) { case Snapshot::') + r'(\w+)' + \
    re.escape(': return reinterpret_cast<RawObject**>(&ptr()->') + r'([a-zA-Z0-9_]+)_' + \
    re.escape('); case Snapshot::') + r'(\w+)' + \
    re.escape(': return reinterpret_cast<RawObject**>(&ptr()->') + r'([a-zA-Z0-9_]+)_' + \
    re.escape('); case Snapshot::') + r'(\w+)' + \
    re.escape(': return reinterpret_cast<RawObject**>(&ptr()->') + r'([a-zA-Z0-9_]+)_' + \
    re.escape('); case Snapshot::kMessage: case Snapshot::kNone: case Snapshot::kInvalid: break; } UNREACHABLE(); return NULL;')

snapshot2_matcher = \
    re.escape('switch (kind) { case Snapshot::') + r'(\w+)' + \
    re.escape(': return reinterpret_cast<RawObject**>(&ptr()->') + r'([a-zA-Z0-9_]+)_' + \
    re.escape('); case Snapshot::') + r'(\w+)' + \
    re.escape(': case Snapshot::') + r'(\w+)' + \
    re.escape(': return reinterpret_cast<RawObject**>(&ptr()->') + r'([a-zA-Z0-9_]+)_' + \
    re.escape('); case Snapshot::kMessage: case Snapshot::kNone: case Snapshot::kInvalid: break; } UNREACHABLE(); return NULL;')

snapshot3_matcher = \
    re.escape('switch (kind) { case Snapshot::') + r'(\w+)' + \
    re.escape(': case Snapshot::') + r'(\w+)' + \
    re.escape(': return reinterpret_cast<RawObject**>(&ptr()->') + r'([a-zA-Z0-9_]+)_' + \
    re.escape('); case Snapshot::') + r'(\w+)' + \
    re.escape(': return reinterpret_cast<RawObject**>(&ptr()->') + r'([a-zA-Z0-9_]+)_' + \
    re.escape('); case Snapshot::kMessage: case Snapshot::kNone: case Snapshot::kInvalid: break; } UNREACHABLE(); return NULL;')

def parse_fields(fields):
    fs = []
    for f in fields:
        if not f.split('//')[0].strip():
            if len(fs) > 0 and (fs[-1][-1] != None):
                i = f.find('//')
                if i != -1: fs[-1][-1] += '\n' + f[i+2:]
            continue
        m = re.fullmatch(r'Raw(\w+)\*\s*([a-zA-Z0-9_]+)_\;(\s*//\s*(.*))?', f.strip())
        if not m:
            #print(f)
            return
        fs.append([m.group(1), m.group(2), m.group(4)])
    return fs

In [275]:
fields = {}
mappings = {}

def parse_entry(e):
    if 'has_preproc' in e:
        print('skipping {} because of preproc'.format(e['kind']))
        return
    #print(e['kind'])
    fs = parse_fields(e['fields'])
    if not fs:
        print('skipping {} because of non-matching fields'.format(e['kind']))
        return
    if e['from'] == 'type_test_stub':
        fs.insert(0, ['Code', 'type_test_stub', None])
    else:
        #print(e['kind'], e['from'], fs[0] if len(fs) else None)
        if len(fs) == 0 or e['from'] != fs[0][1]:
            print('skipping {} because of non-matching from'.format(e['kind']))
            return
    
    if 'snapshot' not in e:
        mappings[e['kind']] = False
    elif e['snapshot'].strip() == 'return to();' or e['snapshot'].strip() == 'return to()':
        pass
    elif e['snapshot'].strip() == 'return reinterpret_cast<RawObject**>(&ptr()->constants_table_);':
        idx = next(filter(lambda x: x[1][1] == 'constants_table', enumerate(fs)))[0]
        fs = fs[:idx+1]
    else:
        snapshot = re.sub(r'\s+', ' ', e['snapshot'].strip())
        m = re.fullmatch(snapshot_matcher, snapshot)
        if m:
            gr = m.groups()
        else:
            m = re.fullmatch(snapshot2_matcher, snapshot)
            if m:
                gr = list(m.groups())
                gr.insert(3, gr[4])
            else:
                m = re.fullmatch(snapshot3_matcher, snapshot)
                if m:
                    gr = list(m.groups())
                    gr.insert(1, gr[2])
                else:
                    print('Couldn\'t match:', snapshot)
                    return
        gr = dict( gr[i*2:i*2+2] for i in range(len(gr) // 2) )
        mappings[e['kind']] = ( gr['kFull'], gr['kFullJIT'], gr['kFullAOT'] )
    fields[e['kind']] = fs

for e in entries: parse_entry(e)

skipping Function because of preproc
skipping Field because of preproc
skipping Code because of preproc
skipping Bytecode because of preproc
skipping LocalVarDescriptors because of non-matching fields
skipping Context because of non-matching fields
skipping ICData because of non-matching fields
skipping TypeArguments because of non-matching fields
skipping String because of preproc
skipping TypedData because of non-matching fields
skipping TypedDataView because of non-matching from
skipping Array because of non-matching fields
skipping ExternalTypedData because of non-matching fields
skipping RegExp because of non-matching fields


In [276]:
def format_fields(fs):
    return "".join("        {},\n".format(repr(x)) for x in fs)
def format_type(e):
    return '    {}: [\n{}    ],\n'.format(repr(e[0]), format_fields(e[1]))
print("".join(format_type(e) for e in fields.items()))

    'Class': [
        ['String', 'name', None],
        ['String', 'user_name', None],
        ['Array', 'functions', None],
        ['Array', 'functions_hash_table', None],
        ['Array', 'fields', None],
        ['Array', 'offset_in_words_to_field', None],
        ['Array', 'interfaces', 'Array of AbstractType.'],
        ['Script', 'script', None],
        ['Library', 'library', None],
        ['TypeArguments', 'type_parameters', 'Array of TypeParameter.'],
        ['AbstractType', 'super_type', None],
        ['Function', 'signature_function', 'Associated function for typedef class.'],
        ['Array', 'constants', 'Canonicalized const instances of this class.'],
        ['Type', 'declaration_type', 'Declaration type for this class.'],
        ['Array', 'invocation_dispatcher_cache', 'Cache for dispatcher functions.'],
        ['Code', 'allocation_stub', 'Stub code for allocation of instances.'],
        ['GrowableObjectArray', 'direct_implementors', 'Array of Class.'],
      

### Results & manual parsing

In [277]:
import subprocess
import yapf

x = subprocess.check_output('xclip -sel clip -o', shell=True).decode()
fs = parse_fields(x.splitlines())

#subprocess.check_output('xclip -sel clip', shell=True, input=l.lstrip().encode())
print(format_fields(fs).lstrip())

['Class', 'object_class', None],
        ['Type', 'object_type', None],
        ['Class', 'null_class', None],
        ['Type', 'null_type', None],
        ['Type', 'function_type', None],
        ['Type', 'type_type', None],
        ['Class', 'closure_class', None],
        ['Type', 'number_type', None],
        ['Type', 'int_type', None],
        ['Class', 'integer_implementation_class', None],
        ['Type', 'int64_type', None],
        ['Class', 'smi_class', None],
        ['Type', 'smi_type', None],
        ['Class', 'mint_class', None],
        ['Type', 'mint_type', None],
        ['Class', 'double_class', None],
        ['Type', 'double_type', None],
        ['Type', 'float32x4_type', None],
        ['Type', 'int32x4_type', None],
        ['Type', 'float64x2_type', None],
        ['Type', 'string_type', None],
        ['TypeArguments', 'type_argument_int', None],
        ['TypeArguments', 'type_argument_double', None],
        ['TypeArguments', 'type_argument_string', None],
 